In [2]:
!pip install librosa

In [3]:
import os
import numpy as np
import librosa
import librosa.display
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, TimeDistributed, Reshape

In [4]:
# Load dataset path
data_path = r"C:\Users\Monish Kiran\Desktop\Monish Kiran\Amrita\Hackathon\IIsc Banglore\Respiratory disease prediction\Respiratory_Sound_Database\Respiratory_Sound_Database"
audio_path = r"C:\Users\Monish Kiran\Desktop\Monish Kiran\Amrita\Hackathon\IIsc Banglore\Respiratory disease prediction\Respiratory_Sound_Database\Respiratory_Sound_Database\audio_and_txt_files"
metadata_file = r"C:\Users\Monish Kiran\Desktop\Monish Kiran\Amrita\Hackathon\IIsc Banglore\Respiratory disease prediction\Respiratory_Sound_Database\Respiratory_Sound_Database\patient_diagnosis.csv"

In [5]:
import pandas as pd
df = pd.read_csv(metadata_file, header=None)
print(df.head())  # Display first few rows
print(df.shape)   # Check number of rows and columns

     0        1
0  101     URTI
1  102  Healthy
2  103   Asthma
3  104     COPD
4  105     URTI
(126, 2)


In [6]:
files = os.listdir(audio_path)
print(files[:10])  # Print first 10 files to verify .wav files exist

['.ipynb_checkpoints', '101_1b1_Al_sc_Meditron.txt', '101_1b1_Al_sc_Meditron.wav', '101_1b1_Pr_sc_Meditron.txt', '101_1b1_Pr_sc_Meditron.wav', '102_1b1_Ar_sc_Meditron.txt', '102_1b1_Ar_sc_Meditron.wav', '103_2b2_Ar_mc_LittC2SE.txt', '103_2b2_Ar_mc_LittC2SE.wav', '104_1b1_Al_sc_Litt3200.txt']


In [7]:
sample_wav = "101_1b1_Al_sc_Meditron.wav"  # Replace with actual file name
extracted_id = sample_wav.split("_")[0]

csv_patient_ids = df[0].astype(str).tolist()  # Convert to string for comparison
print(f"Extracted ID: {extracted_id}")
print(f"Is ID in CSV? {'Yes' if extracted_id in csv_patient_ids else 'No'}")


Extracted ID: 101
Is ID in CSV? Yes


In [8]:
#dataset_path = "Respiratory_Sound_Database/Respiratory_Sound_Database"
#metadata_file = os.path.join(dataset_path, "patient_diagnosis.csv")

# Now open the file correctly
with open(metadata_file, "r") as file:
    for line in file.readlines():
        parts = line.strip().split(",")
        print(parts)  # Check if it's reading correctly


['101', 'URTI']
['102', 'Healthy']
['103', 'Asthma']
['104', 'COPD']
['105', 'URTI']
['106', 'COPD']
['107', 'COPD']
['108', 'LRTI']
['109', 'COPD']
['110', 'COPD']
['111', 'Bronchiectasis']
['112', 'COPD']
['113', 'COPD']
['114', 'COPD']
['115', 'LRTI']
['116', 'Bronchiectasis']
['117', 'COPD']
['118', 'COPD']
['119', 'URTI']
['120', 'COPD']
['121', 'Healthy']
['122', 'Pneumonia']
['123', 'Healthy']
['124', 'COPD']
['125', 'Healthy']
['126', 'Healthy']
['127', 'Healthy']
['128', 'COPD']
['129', 'URTI']
['130', 'COPD']
['131', 'URTI']
['132', 'COPD']
['133', 'COPD']
['134', 'COPD']
['135', 'Pneumonia']
['136', 'Healthy']
['137', 'URTI']
['138', 'COPD']
['139', 'COPD']
['140', 'Pneumonia']
['141', 'COPD']
['142', 'COPD']
['143', 'Healthy']
['144', 'Healthy']
['145', 'COPD']
['146', 'COPD']
['147', 'COPD']
['148', 'URTI']
['149', 'Bronchiolitis']
['150', 'URTI']
['151', 'COPD']
['152', 'Healthy']
['153', 'Healthy']
['154', 'COPD']
['155', 'COPD']
['156', 'COPD']
['157', 'COPD']
['158', '

In [9]:
# Extract MFCC features
def extract_features(file_path, max_pad_len=400):
    signal, sr = librosa.load(file_path, sr=22050)
    mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=40)
    pad_width = max_pad_len - mfcc.shape[1]
    if pad_width > 0:
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_pad_len]
    return mfcc

In [10]:
X, y = [], []
for file in os.listdir(audio_path):
    if file.endswith(".wav"):
        patient_id = file.split("_")[0]
        if str(patient_id) in df[0].astype(str).values:  # Ensure string comparison
            mfcc = extract_features(os.path.join(audio_path, file))
            X.append(mfcc)
            y.append(df[df[0].astype(str) == str(patient_id)][1].values[0])  # Get diagnosis


C:\Users\Monish Kiran\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\Monish Kiran\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\Monish Kiran\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [11]:
# Convert to numpy array
X = np.array(X)
y = np.array(y)

In [12]:
# Encode labels
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = to_categorical(y)

In [13]:
# Reshape data for CNN-LSTM
X = X[..., np.newaxis]  # Add channel dimension
X = np.transpose(X, (0, 2, 1, 3))  # Reshape for LSTM processing
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])

In [14]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
print("X shape:", X.shape)

X shape: (920, 400, 40)


In [16]:
possible_size = int(np.sqrt(X.shape[-1]))  # Try to infer width/height
print("Possible image size:", possible_size)

Possible image size: 6


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed

# Define CNN-LSTM Model
model = Sequential([
    TimeDistributed(Conv1D(32, 3, activation='relu', padding='same'), input_shape=(X.shape[1], X.shape[2], 1)),  
    TimeDistributed(MaxPooling1D(2)),
    TimeDistributed(Flatten()),
    LSTM(64, return_sequences=True),
    LSTM(64),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(y.shape[1], activation='softmax')
])

model.summary()


C:\Users\Monish Kiran\anaconda3\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 400, 40, 32)    │           128 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 400, 20, 32)    │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 400, 640)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 400, 64)        │       180,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 215,976 (843.66 KB)

 Trainable params: 215,976 (843.66 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
# Train model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 432s 17s/step - accuracy: 0.5985 - loss: 1.3890 - val_accuracy: 0.8152 - val_loss: 0.8608
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 342s 15s/step - accuracy: 0.8794 - loss: 0.6615 - val_accuracy: 0.8152 - val_loss: 0.6659
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 342s 15s/step - accuracy: 0.8621 - loss: 0.6500 - val_accuracy: 0.8152 - val_loss: 0.8440
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 388s 17s/step - accuracy: 0.8631 - loss: 0.6786 - val_accuracy: 0.8152 - val_loss: 0.5997
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 386s 17s/step - accuracy: 0.8636 - loss: 0.6610 - val_accuracy: 0.8152 - val_loss: 0.7320
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 380s 16s/step - accuracy: 0.8737 - loss: 0.5487 - val_accuracy: 0.8152 - val_loss: 0.6160
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 371s 16s/step - accuracy: 0.8789 - loss: 0.5073 - val_accuracy: 0.8152 - val_loss: 0.6059
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 348s 15s/step - accuracy: 0.8773 - loss: 0.5612 - val_accuracy: 0.

In [ ]:
# Evaluate model
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")